# Homework - Finite Difference Method

강의에서 바닐라 옵션과 배리어 옵션의 가격결정을 진행해 보았습니다. <br>
이 과제에서는 현재 증권사에서 발행되는 스텝다운 ELS의 가격결정을 진행합니다. <br>

## Step Down ELS

스텝다운 ELS는 다음과 같은 페이오프를 가집니다.
1. 조기상환 일시에 기초자산의 가격이 조기상환 배리어보다 높거나 같은 경우, 초기부터 조기상환 일시까지 약정된 수익률을 지급합니다. <br>
example) 약정금액이 1000, 약정 수익률이 단리 연 10%이고, 만기 $T = 1.5$, 조기상환 기회가 $t = [0.5,1]$ 이고, 조기상환 배리어가 ${S_t\over S_0} >[0.5,0.5]$이면<br>

case1) 초기로부터 반년 (125영업일) 후 기초자산의 가격이 초기의 60%일 때, 조기상환 조건을 충족하므로 $1000 \times (1+0.1 \times 0.5) = 1050$ 지급<br>

case2) 첫 조기상환 조건을 만족시키지 못하고 그로부터 반년 후 기초자산의 가격이 초기의 55%일 때, 조기상환 조건을 충족하므로 $1000 \times (1+0.1 \times 1) = 1100$ 지급 <br>

2. 모든 조기상환 기회에 조기상환 조건을 채우지 못한 경우, 만기상환이 진행됩니다. 만기상환 당시 기초자산의 가격이 원금보장 배리어보다 높은 경우, 초기부터 조기상환 일시까지 약정된 수익률을 지급합니다. 만약 기초자산의 가격이 원금보장 배리어보다 낮은 경우, ${S_T\over S_0} \times$ 약정금액을 지급합니다. 이때 $S_T$는 만기 시점에서의 기초자산의 가격이며, $S_0$는 초기 시점에서의 기초자산의 가격입니다.<br>

example) 약정 수익률이 6%, 만기 $T = 1.5$, 만기 원금보장 조건이 $S_T > 0.5 \times S_0$ 이면
<br>
case1) 조기상환 조건을 만족시키지 못하고 만기시점 기초자산의 가격이 초기의 55%일 때, 원금보장 조건을 충족하므로$1000 \times (1+0.1 \times 1.5) = 1150$ 지급 <br>
case2) 조기상환 조건을 만족시키지 못하고 만기시점 기초자산의 가격이 초기의 45%일 때, 원금보장 조건을 미충족하므로$1000 \times 0.45 = 450$ 지급 <br>

다음 조건하에서 스텝다운 ELS의 현재 가치를 구하시오.
$$S_0 = 100$$
$$\sigma= 0.3$$
<center>약정금액 = 1000</center>  
<center>약정 수익률 = $r_p$=단리 연 12%</center>  
<center>무위험 수익률 = $r$ = 연속 복리 연 2%</center>  
$$T = 3$$
<center>조기상환 기회 : $t = [0.5,1,1.5,2,2.5]$</center>  
<center>조기상환 배리어 : ${S_t \over S_0} \ge [0.9,0.8,0.8,0.7,0.7]$ </center>  
<center>만기 원금보장 조건 : $S_T \ge 0.6 \times S_0$</center>  
<br>
Hint)기존의 FDM과 똑같이 진행하되, 조기상환 기회인 126일차, 251일차, 376일차, 501일차, 751일차 시점에서 기초자산의 가격이 조기상환 배리어보다 높은 경우일 때의 파생상품의 가치를 $1000 \times (1+r_p\times t)$로 치환하면 됩니다. <br>
위 조건을 가지는 ELS를 편의를 위해 $T = [0.5,1,1.5,2,2.5,3], {S_t\over S_0} \ge [0.9,0.8,0.8,0.7,0.7,0.6], r_p = 0.12$ 조건을 가지는 ELS라 부르겠습니다.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
def a(S,t,dS,dt,r,sigma) :
    a1 = 0.5*((sigma*S)/dS)**2
    a2 = 0.5*r*S/dS
    return -a1-a2
def b(S,t,dS,dt,r,sigma) :
    b1 = ((sigma*S)/dS)**2
    return r + b1
def c(S,t,dS,dt,r,sigma) :
    c1 = 0.5*((sigma*S)/dS)**2
    c2 = 0.5*r*S/dS
    return -c1+c2
def ai(S,t,dS,dt,r,sigma) :
    return a(S,t,dS,dt,r,sigma) * dt
def bi(S,t,dS,dt,r,sigma) :
    return b(S,t,dS,dt,r,sigma) * dt +1
def ci(S,t,dS,dt,r,sigma) :
    return c(S,t,dS,dt,r,sigma) * dt
def matrix(Smax,t,dS,dt,r,sigma):
    N = int(Smax/dS)
    mat = np.zeros([N,N])
    mat[0,0] = 2*ci(dS,t,dS,dt,r,sigma)+bi(dS,t,dS,dt,r,sigma)
    mat[0,1] = -ci(dS,t,dS,dt,r,sigma)+ai(dS,t,dS,dt,r,sigma)
    for i in range(1,N-1):
        mat[i,i-1] = ci((i+1)*dS,t,dS,dt,r,sigma)
        mat[i,i] = bi((i+1)*dS,t,dS,dt,r,sigma)
        mat[i,i+1] = ai((i+1)*dS,t,dS,dt,r,sigma)
    mat[N-1,N-2] = ci(Smax,t,dS,dt,r,sigma)-ai(Smax,t,dS,dt,r,sigma)
    mat[N-1,N-1] = bi(Smax,t,dS,dt,r,sigma)+2*ai(Smax,t,dS,dt,r,sigma)
    return mat

In [2]:
C = 1000
s0 = 100
sigma = 0.3
r = 0.02
rp = 0.12
T = 3
exercise = [0.5,1,1.5,2,2.5,3]
barrier = [0.9,0.8,0.8,0.7,0.7,0.6]
days = 250
dt = 1/250
dS = 1
mean = np.exp(r+0.5*sigma**2)
var = (np.exp(sigma**2)-1)*(np.exp(2*r+sigma**2))
Smax = np.ceil((mean+3*np.sqrt(var))**3)*100

In [3]:
FDM = pd.DataFrame(index = np.arange(dS,Smax+dS,dS), columns = range(1,2+T*250))
FDM.loc[1:59,751] = FDM.index[:59]*C/100
FDM.loc[60:,751] = C*(1+rp*3)

In [4]:
for i in range(1,751):
    t = (751-i)/250
    v = np.array(FDM[751-i+1])
    FDM[751-i] = np.linalg.inv(matrix(Smax,t,dS,dt,r,sigma))@v
    year = (751-i)/250
    if year in exercise:
        N = exercise.index(year)
        B = barrier[2]*s0
        FDM.loc[B:,751-i] = C*(1+rp*year)

In [5]:
FDM.loc[100,1]

1029.9326520092281

## Knock-In Step Down ELS

낙인 스텝다운 ELS는 보통 다음과 같이 표시됩니다. $T = [0.5,1,1.5,2,2.5,3], {S_t\over S_0} \ge [0.9,0.8,0.8,0.7,0.7,0.6], r_p = 0.12$,  KI = 0.4 <br>
이는 만약 기초자산의 종가가 만기가 도래하기 전 한 번이라도 초기 기초자산의 가격의 40% 미만에 도달한 경우, 상환 배리어가 ${S_t \over S_0} \ge [0.9,0.8,0.8,0.7,0.7,0.4]$ 에서
${S_t \over S_0} \ge [0.9,0.8,0.8,0.7,0.7, 0.6]$로 바뀌는 것을 의미합니다. <br>그 이외 모든 조건은 위 문제와 같을 때, 다음 조건을 가지는 낙인 스텝다운 ELS의 현재 가치를 구하시오.<br>

<br>
<center>$T = [0.5,1,1.5,2,2.5,3], {S_t\over S_0} \ge [0.9,0.8,0.8,0.7,0.7,0.6], r_p = 0.12$,  KI = 0.4<center>
<br>
    
Hint) 배리어 옵션의 가격을 계산했을 때와 마찬가지로, 낙인조건이 발생한 후의 가격격자를 만들고, 낙인조건이 발생하는 영역의 가격을 해당 격자의 가격으로 교체하면 됩니다.

In [6]:
KI = pd.DataFrame(index = np.arange(dS,Smax+dS,dS), columns = range(1,2+T*250))

In [7]:
KI.loc[1:39,751] = KI.index[:39]*C/100
KI.loc[40:,751] = C*(1+rp*3)

In [8]:
for i in range(1,751):
    t = (751-i)/250
    v = np.array(KI[751-i+1])
    KI[751-i] = np.linalg.inv(matrix(Smax,t,dS,dt,r,sigma))@v
    year = (751-i)/250
    if year in exercise:
        N = exercise.index(year)
        B = barrier[2]*s0
        KI.loc[B:,751-i] = C*(1+rp*year)
    KI.loc[1:39,751-i] = FDM.loc[1:39,751-i].copy()

In [9]:
KI.loc[100,1]

1043.4056253315396

낙인 조건을 만족하지 않았을 경우, 위 문제에서 계산한 노낙인 ELS보다 유리하기에 현재가치가 더 높은 것을 볼 수 있습니다.